# Course 2 Assignment 1

For this assignment we had to perform an ANOVA test and a post hoc comparison, then interpret the results

In [1]:
# import needed libraries
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi

### Loading and preparing data

In [2]:
# show all columns
pd.set_option('display.max_columns', None)
# loading the data from the local file
df = pd.read_csv('data/covid_data.csv')

In [3]:
# prepare data
df.date = pd.to_datetime(df.date)
dfx = df.dropna(subset=['continent'])  # gets rid of summaries for 'world' and 'africa' etc, as I only want data for countries
# the columns I need for this task
cols = ['location', 'date', 'new_deaths_per_million', 'people_fully_vaccinated', 'human_development_index', 'population']
dfx = dfx[cols].dropna()  # getting rid of rows with empty data
# getting rid of rows where new deaths are below zero (due to error correction I guess?)
dfx = dfx[dfx.new_deaths_per_million >= 0]
# limiting it to 2021 which is when vaccinations really got started
dfx = dfx[dfx['date'].dt.year == 2021]
# so as to compare like with like, I'm keeping only countries with human development indices over 0.9
dfx = dfx[dfx.human_development_index > 0.9]

In [4]:
# calculating percentage of population fully vaccinated
dfx['percentage_fully_vaccinated'] = (dfx.people_fully_vaccinated/dfx.population) * 100
dfx.head()

,location,date,new_deaths_per_million,people_fully_vaccinated,human_development_index,population,percentage_fully_vaccinated
5245,Austria,2021-01-15,7.328,1.0,0.922,9006400.0,0.000011
5246,Austria,2021-01-16,7.328,10.0,0.922,9006400.0,0.000111
5247,Austria,2021-01-17,3.220,370.0,0.922,9006400.0,0.004108
5248,Austria,2021-01-18,4.441,1261.0,0.922,9006400.0,0.014001
5249,Austria,2021-01-19,8.549,2714.0,0.922,9006400.0,0.030134


#### splitting data into bins to make a categorical variable

In [5]:
dfx.percentage_fully_vaccinated.describe()  # examining data to determine how to split it into bins

count    1854.000000
mean        5.748426
std        10.134611
min         0.000011
25%         1.034173
50%         2.853313
75%         5.814792
max        58.816924
Name: percentage_fully_vaccinated, dtype: float64

In [6]:
# bins mostly based on quartile ranges, but because spread is so wide adding extras at top end
bins = [0, 1, 3, 6, 10, 30, 60]  
dfx['percentage_band'] = pd.cut(dfx.percentage_fully_vaccinated, bins, labels=['less than 1%', '1 to 3%', '3 to 6%', '6 to 10%', '10 to 30%', 'over 30%'])
dfx.tail()

,location,date,new_deaths_per_million,people_fully_vaccinated,human_development_index,population,percentage_fully_vaccinated,percentage_band
81005,United States,2021-04-25,0.843,94772329.0,0.926,331002647.0,28.631895,10 to 30%
81006,United States,2021-04-26,1.432,95888088.0,0.926,331002647.0,28.968979,10 to 30%
81007,United States,2021-04-27,1.937,96747454.0,0.926,331002647.0,29.228604,10 to 30%
81008,United States,2021-04-28,2.897,98044421.0,0.926,331002647.0,29.620434,10 to 30%
81009,United States,2021-04-29,2.580,99668945.0,0.926,331002647.0,30.111223,over 30%


### ANOVA test

In [7]:
# using ols function for calculating the F-statistic and associated p value
model = smf.ols(formula='new_deaths_per_million ~ C(percentage_band)', data=dfx)  # C means variable is categorical
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     new_deaths_per_million   R-squared:                       0.061
Model:                                OLS   Adj. R-squared:                  0.058
Method:                     Least Squares   F-statistic:                     23.81
Date:                    Sat, 01 May 2021   Prob (F-statistic):           2.85e-23
Time:                            15:47:50   Log-Likelihood:                -5114.7
No. Observations:                    1854   AIC:                         1.024e+04
Df Residuals:                        1848   BIC:                         1.027e+04
Df Model:                               5                                         
Covariance Type:                nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           4.5522      0.179     25.445      0.000       4.201       4.903
C(percentage_band)[T.1 to 3%]      -1.3096      0.246     -5.316      0.000      -1.793      -0.826
C(percentage_band)[T.3 to 6%]      -2.3864      0.255     -9.374      0.000      -2.886      -1.887
C(percentage_band)[T.6 to 10%]     -2.3049      0.295     -7.815      0.000      -2.883      -1.726
C(percentage_band)[T.10 to 30%]    -1.7862      0.421     -4.248      0.000      -2.611      -0.961
C(percentage_band)[T.over 30%]     -2.9904      0.479     -6.240      0.000      -3.930      -2.051
==============================================================================
Omnibus:                      866.459   Durbin-Watson:                   0.836
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5310.217
Skew:                           2.133   Prob(JB):                         0.00
Kurtosis:                      10.109   Cond. No.                         6.66
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### printing means and standard deviations

In [8]:
print('means for new deaths per million by percentage vaccinated')
dfx.groupby('percentage_band')['new_deaths_per_million'].mean()

means for new deaths per million by percentage vaccinated


percentage_band
less than 1%    4.552245
1 to 3%         3.242680
3 to 6%         2.165883
6 to 10%        2.247361
10 to 30%       2.766050
over 30%        1.561811
Name: new_deaths_per_million, dtype: float64

In [9]:
print('standard deviations for new deaths per million by percentage vaccinated')
dfx.groupby('percentage_band')['new_deaths_per_million'].std()

standard deviations for new deaths per million by percentage vaccinated


percentage_band
less than 1%    5.414749
1 to 3%         4.059314
3 to 6%         2.533056
6 to 10%        2.542878
10 to 30%       2.416557
over 30%        1.284648
Name: new_deaths_per_million, dtype: float64

### Post hoc comparison

In [10]:
print('Post hoc comparison')
mc = multi.MultiComparison(dfx.new_deaths_per_million, dfx.percentage_band)
res = mc.tukeyhsd()
res.summary()

Post hoc comparison


group1,group2,meandiff,p-adj,lower,upper,reject
1 to 3%,10 to 30%,-0.4766,0.848,-1.6649,0.7116,False
1 to 3%,3 to 6%,-1.0768,0.001,-1.7841,-0.3695,True
1 to 3%,6 to 10%,-0.9953,0.0078,-1.8205,-0.1702,True
1 to 3%,less than 1%,1.3096,0.001,0.6068,2.0123,True
1 to 3%,over 30%,-1.6809,0.0056,-3.038,-0.3237,True
10 to 30%,3 to 6%,-0.6002,0.6865,-1.8024,0.6021,False
10 to 30%,6 to 10%,-0.5187,0.8387,-1.7938,0.7565,False
10 to 30%,less than 1%,1.7862,0.001,0.5866,2.9858,True
10 to 30%,over 30%,-1.2042,0.3102,-2.8737,0.4652,False
3 to 6%,6 to 10%,0.0815,0.9,-0.7637,0.9267,False


## Interpretation

ANOVA revealed that among countries with a Human Development Index of over 0.9 (my sample), the percentage of population vaccinated (collapsed into 6 ordered categories, which is the categorical variable) and the number of new deaths per million people (quantitative response variable) were significantly associated, F(5, 1843)=23.81 p<000.1.

Post hoc comparisons of the mean number of new deaths per million people by pairs of population vaccinated categories showed the following:

The mean new deaths when less than 1% of the population were vaccinated were significantly different from all other means.
The mean new deaths when 1 to 3% of the population were vaccinated were significantly different from all other means, except for the 10 to 30% band.
The other differences in means were not statistically significant.